# MOEX Minute Loader Demo

Используем свежий модуль `moex_ohcl_loader` для автоматической докачки минутных свечей с Московской биржи по тикерам, встречающимся в последнем тесте из файла `SiBDt.csv`, а затем построим эквити с помощью уже существующего конструктора.

In [1]:
from pathlib import Path

import pandas as pd

from src.data.moex_ohcl_loader import download_moex_ohlc
from src.data.equity_builder import build_equity

In [2]:
pd.set_option("display.max_rows", 10)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 120)

log_path = Path("SiBDt.csv")
candles = download_moex_ohlc(log_path)
candles.head()

,DATE_TIME,OPEN,HIGH,LOW,CLOSE,VOL,TICKER,1_step_price
0,2025-07-23 09:22:00,80671.0,80682.0,80667.0,80682.0,56,SiU5,1.0
1,2025-07-23 09:23:00,80684.0,80749.0,80681.0,80746.0,826,SiU5,1.0
2,2025-07-23 09:24:00,80746.0,80828.0,80746.0,80810.0,2653,SiU5,1.0
3,2025-07-23 09:25:00,80811.0,80944.0,80810.0,80934.0,3324,SiU5,1.0
4,2025-07-23 09:26:00,80934.0,80981.0,80933.0,80933.0,3330,SiU5,1.0


In [3]:
candles_path = Path("OHCL") / f"{log_path.stem}_ohcl.csv"
candles_path

WindowsPath('OHCL/SiBDt_ohcl.csv')

In [4]:
equity = build_equity(candles_csv=candles_path, trades_csv=log_path)
equity.head()

c:\Users\user\Documents\spreader_pro\code\test_visualisation\src\data\equity_builder.py:27: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  return pd.to_datetime(col, errors="coerce").dt.floor("T")
c:\Users\user\Documents\spreader_pro\code\test_visualisation\src\data\csv_transformer.py:117: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df[date_col] = df[date_col].dt.floor("T")  # 'T' == minute
c:\Users\user\Documents\spreader_pro\code\test_visualisation\src\data\equity_builder.py:27: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  return pd.to_datetime(col, errors="coerce").dt.floor("T")


,DATE_TIME,OPEN,HIGH,LOW,CLOSE,VOL,TICKER,1_step_price,date_time,action_price,current_pos,prev_position,deal_price,prev_action_price,delta_price,Pnl,Equity
0,2025-07-23 09:22:00,80671.0,80682.0,80667.0,80682.0,56,SiU5,1.0,2025-07-23 09:22:00,80667.0,0,0,NaN,80667.0,0.0,0.0,0.0
1,2025-07-23 09:23:00,80684.0,80749.0,80681.0,80746.0,826,SiU5,1.0,2025-07-23 09:23:00,80744.0,-1,0,80744.0,80667.0,77.0,0.0,0.0
2,2025-07-23 09:24:00,80746.0,80828.0,80746.0,80810.0,2653,SiU5,1.0,2025-07-23 09:24:00,80828.0,-1,-1,NaN,80744.0,84.0,-84.0,-84.0
3,2025-07-23 09:25:00,80811.0,80944.0,80810.0,80934.0,3324,SiU5,1.0,2025-07-23 09:25:00,80944.0,-1,-1,NaN,80828.0,116.0,-116.0,-200.0
4,2025-07-23 09:26:00,80934.0,80981.0,80933.0,80933.0,3330,SiU5,1.0,2025-07-23 09:26:00,80981.0,-1,-1,NaN,80944.0,37.0,-37.0,-237.0


In [6]:
equity.tail()

,DATE_TIME,OPEN,HIGH,LOW,CLOSE,VOL,TICKER,1_step_price,date_time,action_price,current_pos,prev_position,deal_price,prev_action_price,delta_price,Pnl,Equity
65924,2025-11-06 16:33:00,82214.0,82214.0,82181.0,82182.0,248,SiZ5,1.0,2025-11-06 16:33:00,82181.0,4,4,NaN,82205.0,-24.0,-96.0,39002.46
65925,2025-11-06 16:34:00,82184.0,82184.0,82150.0,82168.0,659,SiZ5,1.0,2025-11-06 16:34:00,82156.0,5,4,82156.0,82181.0,-25.0,-100.0,38902.46
65926,2025-11-06 16:35:00,82167.0,82183.0,82126.0,82130.0,1444,SiZ5,1.0,2025-11-06 16:35:00,82126.0,5,5,NaN,82156.0,-30.0,-150.0,38752.46
65927,2025-11-06 16:36:00,82130.0,82167.0,82120.0,82160.0,975,SiZ5,1.0,2025-11-06 16:36:00,82126.0,-5,5,82126.0,82126.0,0.0,0.0,38752.46
65928,2025-11-06 16:37:00,82160.0,82198.0,82153.0,82184.0,527,SiZ5,1.0,2025-11-06 16:37:00,82198.0,-5,-5,NaN,82126.0,72.0,-360.0,38392.46


In [5]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scattergl(        # ⚡ ScatterGL — рендер на GPU
    x=equity['DATE_TIME'],
    y=equity['Equity'],
    mode='lines',
    line=dict(width=1, color='blue')
))
fig.show()